In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from pygpt4all.models.gpt4all_j import GPT4All_J
import time 
from datetime import datetime

In [2]:


def getanswer(question):
    def new_text_callback(text):
        print(text, end='')
        return text
    model = GPT4All_J('/Users/aaronsteiner/Downloads/ggml-gpt4all-j-v1.2-jazzy.bin')
    response = model.generate(question, seed=42069, new_text_callback=new_text_callback, n_threads=4, n_batch=20)
    return response.replace(question, '').strip()

In [3]:
getanswer("product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer Men's Special Edition Heuer Monaco Watch Chrono Grapple 11 Automatic. Do these match exactly? Answer with yes or no. If the materials or colors do not match consider it a non match")

gptj_model_load: loading model from '/Users/aaronsteiner/Downloads/ggml-gpt4all-j-v1.2-jazzy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 4505.45 MB
gptj_model_load: memory_size =   896.00 MB, n_mem = 57344
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285
gptj_generate: seed = 42069
gptj_generate: number of tokens in prompt = 65

product 1: TAG Heuer Monaco Chronograph Calibre 11 Automatic, product 2: TAG Heuer Men's Special Edition Heuer Monaco Watch Chrono Grapple 11 Automatic. Do these match exactly? Answer with yes or no. If the materials or colors do not match consider it a non match.<|endoftext|>

gptj_generate: mem per token = 15473904 bytes
gptj_generate:     load time =     0.00 ms
gp

'.<|endoftext|>'

## Start Testing

In [4]:
# open our test dataframe from the csv
df = pd.read_csv('data/test/random_sample_100_v2.csv')
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,priceCurrency_right,specTableContent_right,cluster_id_right,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction
0,70108616,Maxxis,Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead,The new incarnation of the Minion DHR. Ready t...,110.00,CAD,NaN,1084991,45040021,Maxxis,...,USD,NaN,1084991,70108616#45040021,1,False,"[-4.3633804321, 4.6556377411]",1,[1.0],1
1,67977820,NaN,Canon CLI251XL High Yield Black Inkjet Cartrid...,Crazy Inkjets is a leading supplier of high qu...,6.95,USD,NaN,767463,69787972,Samsung,...,AUD,NaN,1467531,67977820#69787972,0,True,"[3.8366084099, -4.2942962646]",0,[0.0],0
2,79859336,Crucial,Crucial Crucial Memory 4GB DDR4 2666 Unbuffere...,NaN,34.99,CAD,NaN,1892167,65646040,Crucial,...,TRY,NaN,1892167,79859336#65646040,1,False,"[-4.363576889, 4.6508393288]",1,[1.0],1
3,30368884,NaN,"Crucial MX500 250GB 2.5"" SATA III","MX500 250GB SATA 2.5-inch, SATA 6.0Gb/s, 560 M...",43.59,EUR,NaN,672125,86893846,NaN,...,ZAR,NaN,1409979,30368884#86893846,0,False,"[3.8347194195, -4.2961273193]",0,[0.0],0
4,82078171,NaN,Intel Core I3 7th Gen 7100 3.90 Ghz; 2 Core 4 ...,"ntel i3-7100, Core. Processor family: 7th gen ...",2.899E3,ZAR,NaN,443612,6914049,WESTERN DIGITAL,...,EUR,NaN,549556,82078171#6914049,0,False,"[3.8386721611, -4.3010449409]",0,[0.0],0


In [5]:
# create a new column for the chatbot response
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [6]:
def clean_response(response):
    if "yes" in response.lower():
        return 1
    elif "no" in response.lower():
        return 0
    else:
        return -1

In [7]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = f"Do the following two product decriptions match. Answer with yes or no. product 1: {row['title_left']}, product 2: {row['title_right']}"
    error = ""
    
    for i in range(4):
        try:
            response = getanswer(message)
        except:
            error = "Error: response was not set successfully"
            print(error)
            response = ""
        print(response)
        if clean_response(response) != -1:
            break
        else:
            print("Again")

    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/100 [00:00<?, ?it/s]

gptj_model_load: loading model from '/Users/aaronsteiner/Downloads/ggml-gpt4all-j-v1.2-jazzy.bin' - please wait ...
Do the following two product decriptions match. Answer with yes or no. product 1:gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 4505.45 MB
gptj_model_load: memory_size =   896.00 MB, n_mem = 57344
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285
gptj_generate: seed = 42069
gptj_generate: number of tokens in prompt = 83

 Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead, product 2: Maxxis Minion DHR II 3C MaxxTerra/DD TR 29\" Tire - 29 x 2.3\" (Folding Bead)
Product 1: Maxxis Maxxis Minion DHR2 29 x 2.3 Foldiing Bed
Product 2: Maxxis Minion DHR II 3C MaxxTerra/DD TR 29\" Tire - 29 x 2.3\" (Foldi ng BeaD)
Y

: 

: 

In [ ]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response_raw'].apply(lambda x: clean_response(x))
df.head()

# Check the accuracy of the chatbot 
df[df['chatbot_response_clean'] == df['label']].shape[0]
print(f"Accuracy: {df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]}")
# Check the number of -1 responses
df[df['chatbot_response_clean'] == -1].shape[0]

Accuracy: 0.35


36

In [ ]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/gpt4all/new_random_sampel/{now}_simple_promt.json')

In [ ]:
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,70108616,Maxxis,Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead,The new incarnation of the Minion DHR. Ready t...,110.00,CAD,NaN,1084991,45040021,Maxxis,...,70108616#45040021,1,False,"[-4.3633804321, 4.6556377411]",1,[1.0],1,Product 1: Maxxis Maxxis Minion DHR2 29 x 2.3 ...,Do the following two product decriptions match...,1
1,67977820,NaN,Canon CLI251XL High Yield Black Inkjet Cartrid...,Crazy Inkjets is a leading supplier of high qu...,6.95,USD,NaN,767463,69787972,Samsung,...,67977820#69787972,0,True,"[3.8366084099, -4.2942962646]",0,[0.0],0,- All are printer cartridges.<|endoftext|>,Do the following two product decriptions match...,-1
2,79859336,Crucial,Crucial Crucial Memory 4GB DDR4 2666 Unbuffere...,NaN,34.99,CAD,NaN,1892167,65646040,Crucial,...,79859336#65646040,1,False,"[-4.363576889, 4.6508393288]",1,[1.0],1,Product 1: Crucial Crucial Memory 4GB DDR4 266...,Do the following two product decriptions match...,-1
3,30368884,NaN,"Crucial MX500 250GB 2.5"" SATA III","MX500 250GB SATA 2.5-inch, SATA 6.0Gb/s, 560 M...",43.59,EUR,NaN,672125,86893846,NaN,...,30368884#86893846,0,False,"[3.8347194195, -4.2961273193]",0,[0.0],0,<|endoftext|>,Do the following two product decriptions match...,-1
4,82078171,NaN,Intel Core I3 7th Gen 7100 3.90 Ghz; 2 Core 4 ...,"ntel i3-7100, Core. Processor family: 7th gen ...",2.899E3,ZAR,NaN,443612,6914049,WESTERN DIGITAL,...,82078171#6914049,0,False,"[3.8386721611, -4.3010449409]",0,[0.0],0,0T0C0B; 1-2 SATA III 6Gb/s; 64 GB; SLC 0.5 Mb ...,Do the following two product decriptions match...,1


In [ ]:
# Look at the wrong predictions
df[df['chatbot_response_clean'] != df['label']]

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response_raw,chatbot_question,chatbot_response_clean
1,67977820,NaN,Canon CLI251XL High Yield Black Inkjet Cartrid...,Crazy Inkjets is a leading supplier of high qu...,6.95,USD,NaN,767463,69787972,Samsung,...,67977820#69787972,0,True,"[3.8366084099, -4.2942962646]",0,[0.0],0,- All are printer cartridges.<|endoftext|>,Do the following two product decriptions match...,-1
2,79859336,Crucial,Crucial Crucial Memory 4GB DDR4 2666 Unbuffere...,NaN,34.99,CAD,NaN,1892167,65646040,Crucial,...,79859336#65646040,1,False,"[-4.363576889, 4.6508393288]",1,[1.0],1,Product 1: Crucial Crucial Memory 4GB DDR4 266...,Do the following two product decriptions match...,-1
3,30368884,NaN,"Crucial MX500 250GB 2.5"" SATA III","MX500 250GB SATA 2.5-inch, SATA 6.0Gb/s, 560 M...",43.59,EUR,NaN,672125,86893846,NaN,...,30368884#86893846,0,False,"[3.8347194195, -4.2961273193]",0,[0.0],0,<|endoftext|>,Do the following two product decriptions match...,-1
4,82078171,NaN,Intel Core I3 7th Gen 7100 3.90 Ghz; 2 Core 4 ...,"ntel i3-7100, Core. Processor family: 7th gen ...",2.899E3,ZAR,NaN,443612,6914049,WESTERN DIGITAL,...,82078171#6914049,0,False,"[3.8386721611, -4.3010449409]",0,[0.0],0,0T0C0B; 1-2 SATA III 6Gb/s; 64 GB; SLC 0.5 Mb ...,Do the following two product decriptions match...,1
5,38366111,NaN,Tissot Seastar 1000 T120.417.11.051.00,NaN,525.00,EUR,NaN,1508649,62268972,NaN,...,38366111#62268972,0,False,"[3.8253262043, -4.289003849]",0,[0.0],0,"0 - 32 GB - Black - $60.00, $99.00.\nYes, thes...",Do the following two product decriptions match...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,13807745,Western Digital,Western Digital Blue PC SSD 250GB,Western Digital Blue PC SSD 250GB – PC Case Ge...,135,AUD,NaN,483248,62156844,Western Digital,...,13807745#62156844,1,False,"[-4.3939409256, 4.6775622368]",1,[1.0],1,-00.<|endoftext|>,Do the following two product decriptions match...,-1
95,53525303,Ryze,Ryze Tello Battery Charging Hub G1CH,NaN,129,DKK,NaN,1555405,79449686,NaN,...,53525303#79449686,0,True,"[3.6632392406000003, -4.1180138588]",0,[0.0],0,"G5CQ\nYes, the product descriptions for Ryze T...",Do the following two product decriptions match...,1
97,53053823,NaN,Canon EF-S 18-135mm f/3.5-5.6 IS USM Zoom Lens,The Canon EF-S 18-135mm f/3.5-5.6 IS USM Zoom ...,799,AUD,NaN,390091,45511817,NaN,...,53053823#45511817,0,True,"[3.7480974197, -4.2194314003]",0,[0.0],0,<|endoftext|>,Do the following two product decriptions match...,-1
98,72039756,NaN,"Cooler Master Chassis, MasterBox Lite 5 Tower",NaN,64.52,USD,NaN,1006147,43470746,NaN,...,72039756#43470746,1,False,"[-4.3535614014, 4.6426811218]",1,[1.0],1,OKI.<|endoftext|>,Do the following two product decriptions match...,-1


In [ ]:
# unique value counts of the chatbot response
df['chatbot_response_clean'].value_counts()

-1    36
 1    34
 0    30
Name: chatbot_response_clean, dtype: int64